# **Import Google Drive**


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Setting**


In [26]:
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics
from scipy.stats import levene
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# **Function: Partition all submissions according to "Greedy" and "Multi-owner" methods**


In [27]:
def validate(partition, graph, n):
	# check that
	# 1. each paper is in exactly one partition
	# 2. the number of papers in all partition is equal to the number of papers
	# 3. each partition has at least two authors owns all papers in the partition

	papers = set()
	full_author_parts = []
	for part in partition[:-1]:
		assert( len(part.intersection(papers)) == 0 )
		papers |= part
		# find all authors that can rank all papers in this part
		cnt = 0
		author_part = set()
		for author, val in graph.items():
			if len(val.intersection(part)) == len(part):
				cnt += 1
				author_part.add(author)

		# assert( cnt >= 2 )
		full_author_parts.append(author_part)

	# add the last partition
	full_author_parts.append(set())
	papers |= partition[-1]
	assert(len(papers) == n)

	return full_author_parts


def greedy(graph, m, n, randomize=False, level=1):
	partition = []
	author_parts = []
	allocated_papers = set()

	if level == 1:
		parts = [ graph[i].copy() for i in range(m) ]
		index2pair = [ set([i]) for i in range(m) ]
	elif level == 2:
		parts = [ graph[i].intersection(graph[j])  for i in range(m) for j in range(i+1, m) ]
		index2pair = [ set([i,j]) for i in range(m) for j in range(i+1, m) ]

	max_idx = 0
	max_val = 0
	active_indices = set( [i for i in range(len(parts)) if len(parts[i]) > 1] )
	# for i in range(0, len(parts)):
	for i in active_indices:
		if len(parts[i]) > max_val:
			max_idx = i
			max_val = len(parts[i])

	while len(allocated_papers) < n and len(parts[max_idx]) > 1:
		max_part = parts[max_idx].copy()
		partition.append( max_part )
		author_parts.append( index2pair[max_idx] )
		allocated_papers |= max_part

		max_idx = 0
		max_val = 0
		# for i in range(0, len(parts)):
		to_remove = set()
		for i in active_indices:
			parts[i].difference_update(max_part)
			if len(parts[i]) < 2:
				to_remove.add(i)
				continue
			if len(parts[i]) > max_val:
				max_idx = i
				max_val = len(parts[i])
		max_part = parts[max_idx].copy()
		active_indices.difference_update(to_remove)

	# add all remaining papers to the last partition
	partition.append(set())
	author_parts.append(set())
	for i in range(n):
		if i not in allocated_papers:
			partition[-1].add(i)

	return partition, author_parts


def arbitrary(graph, m, n, randomize=False):
	partition = []
	author_parts = []
	allocated_papers = set()

	parts = [ graph[i].copy() for i in range(m) ]
	index2pair = [ set([i]) for i in range(m) ]

	idx = 0
	while len(parts[idx]) < 2 and idx < len(parts)-1: idx += 1

	while len(allocated_papers) < n and idx != -1:
		part = parts[idx].copy()

		partition.append( part )
		author_parts.append( index2pair[idx] )
		allocated_papers |= part

		idx = -1
		val = None
		for i in range(0, len(parts)):
			parts[i].difference_update(part)
			if len(parts[i]) >= 2:
				idx = i
				val = len(parts[i])

	partition.append(set())
	author_parts.append(set())
	for i in range(n):
		if i not in allocated_papers:
			partition[-1].add(i)

	return partition, author_parts


# **Greedy Isotonic Scores, Proxy**

## Load CSV file into a pandas DataFrame


In [ ]:
# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'proxy_score.csv')
df = df.drop_duplicates(['submission_idx', 'author_idx'])


# Extract the unique authors from the DataFrame
authors = df['author_idx'].unique()
submissions = df['submission_idx'].unique()

author_submission = {}
for author in authors:
    submissionss = list( set(df[df['author_idx'] == author]['submission_idx'].tolist()) )
    author_submission[author] = submissionss

m_2 = len(author_submission)
n_2 = len(submissions)

## Organize all the authors and submissions as the following 'graph': authors = [..., {..., paper_idx : ranking, ...}, ...]


In [29]:
authors = []
for author in author_submission:
    submission_ranking = {}
    for i in range(len(author_submission[author])):
        submission_ranking[author_submission[author][i]] = df[(df['submission_idx'] == author_submission[author][i]) & (df['author_idx'] == author)]['rank'].tolist()
        submission_ranking[author_submission[author][i]] = submission_ranking[author_submission[author][i]][0]
    authors.append(submission_ranking)

graph = {}
for i, author in enumerate(authors):
    graph[i] = set( int(k) for k in author.keys())



## Greedy Isotonic


In [30]:
# Partition all the graph according to greedy algorithm
partition, author_parts = greedy(graph, m_2, n_2)
author_parts = validate(partition, graph, n_2)

calibrated_scores = np.zeros(n_2)
for part, author_part in zip(partition, author_parts):
  if len(author_part) == 0:
    for i in part: calibrated_scores[i] = df[df['submission_idx'] == i]['rating_0312_mean'].tolist()[0]
    continue
  paper_part = list(part)



  # Organize each block by {author: [submission, rank, score]}.
  author_submission_rank_old = {}
  for author in author_part:
    author_submission_rank_old[author] = []
    for i in range(len(paper_part)):
        rank = df[(df['submission_idx'] == paper_part[i]) & (df['author_idx'] == author)]['rank'].tolist()[0]
        ratings = df[(df['submission_idx'] == paper_part[i]) & (df['author_idx'] == author)]['rating_0312_mean'].tolist()[0]
        author_submission_rank_old[author].append((paper_part[i], rank, ratings))



  # Sort submissions by rank; in case of ties, sort by score.
  def sort_submissions(author_submission_rank_old):
    for author in author_submission_rank_old:
      author_submission_rank_old[author].sort(key=lambda x: (x[1], -x[2]), reverse=False)
    return author_submission_rank_old
  author_submission_rank_old = sort_submissions(author_submission_rank_old)



  # Compute isotonic scores for each author in the block.
  author_submission_rank_greedy_iso = {}
  for author in author_submission_rank_old:
      ir_rank = []
      for i in range(len(author_submission_rank_old[author])):
          r1 = author_submission_rank_old[author][i][2]
          ir_rank.append(r1)
      ir_rank = np.array(ir_rank)
      ir_rank_pred =  isotonic_regression(ir_rank, sample_weight = None, y_min=0.0, y_max=10.0, increasing=False)

      author_submission_rank_greedy_iso[author] = []
      for i in range(len(author_submission_rank_old[author])):
          author_submission_rank_greedy_iso[author].append((author_submission_rank_old[author][i][0], author_submission_rank_old[author][i][1], ir_rank_pred[i]))



  # For multi-author submissions in a block, average the scores to get the isotonic score.
  submission_greedy_iso_rating = {}
  for submission in paper_part:
      submission_greedy_iso_rating[submission] = []

  for author in author_submission_rank_greedy_iso:
      for i in range(len(author_submission_rank_greedy_iso[author])):
          submission_greedy_iso_rating[author_submission_rank_greedy_iso[author][i][0]].append(author_submission_rank_greedy_iso[author][i][2])

  for submission in submission_greedy_iso_rating:
      submission_greedy_iso_rating[submission] = [float(rating) for rating in submission_greedy_iso_rating[submission]]
      avg_rating = sum(submission_greedy_iso_rating[submission])/len(submission_greedy_iso_rating[submission])
      submission_greedy_iso_rating[submission] = avg_rating

  for i in paper_part:
    calibrated_scores[i] = submission_greedy_iso_rating[i]



# Greedy Isotonic Score
greedy_iso_rating = {}
for i in range(n_2):
  greedy_iso_rating[i] = calibrated_scores[i]



## Compute MSE values for all submissions.

In [31]:
# proxy
submission_true_rating = {}
for submission in range(n_2):
    submission_true_rating[submission] = df[df['submission_idx'] == submission]['rating_0312_mean'].tolist()
    submission_true_rating[submission] = submission_true_rating[submission][0]

True_score_multi_iso = []
for i in range(n_2):
    True_score_multi_iso.append(submission_true_rating[i])



# Overlap Rate

In [32]:
Percentage = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
adhoc_overlap_percentage = []
multi_overlap_percentage = []
greedy_overlap_percentage = []
for i in range(len(Percentage)):
  percent = Percentage[i]
  # Find the top 10% values in old_rating
  top_x_greedy = sorted(greedy_iso_rating, key=greedy_iso_rating.get, reverse=True)[:int(len(greedy_iso_rating)* percent)]

  # Find the top 10% values in old_rating
  top_x_true = sorted(submission_true_rating, key=submission_true_rating.get, reverse=True)[:int(len(submission_true_rating)* percent)]

  top_x_greedy = set(top_x_greedy)
  top_x_true = set(top_x_true)

  greedy_overlap = top_x_greedy & top_x_true
  # Count the number of overlapping elements
  greedy_overlap_count = len(greedy_overlap)
  greedy_overlap_percentage.append(greedy_overlap_count/len(top_x_true))

greedy_overlap_percentage

[0.7063492063492064,
 0.691699604743083,
 0.7308707124010554,
 0.7806324110671937,
 0.7753164556962026,
 0.8326745718050066]